## Algorytmy Tekstowe - laboratorium 6

In [195]:
import numpy as np
import math
import sys
import os
import pickle

##### Przydatne funkcje
- unique_char - znajduje znak niewystępujący w tekście
- counting_sort - sortowanie przez zliczanie

In [129]:
def unique_char(text):
    i = ord('!')
    while chr(i) in text:
        i += 1
    return chr(i)

def counting_sort(values, key=lambda v: v):
    max_key = key(max(values, key=key))
    counter = np.zeros(ord(max_key) + 1, dtype=np.int32)
    unique = 0
    for val in values:
        if counter[ord(key(val))] == 0:
            unique += 1
        counter[ord(key(val))] += 1
    summed = np.zeros(ord(max_key) + 1, dtype=np.int32)
    for i in range(1,ord(max_key) + 1):
        summed[i] = summed[i-1] + counter[i-1]
    
    final_list = [None] * len(values)
    for val in values:
        final_list[summed[ord(key(val))]] = val
        summed[ord(key(val))] += 1
    
    return final_list

### Funkcja sort-rename

In [130]:
def sort_rename(sequence, sorting=sorted):
    last_entry = None
    index = 0
    position_to_index = [None] * len(sequence)
    first_entry = {}
    for entry in sorting([(e,i) for i, e in enumerate(sequence)], key=lambda v: v[0]):
        if last_entry and last_entry[0] != entry[0]:
            index += 1
            first_entry[index] = entry[1]
        
        position_to_index[entry[1]] = index
        if last_entry is None:
            first_entry[0] = entry[1]
        last_entry = entry
    
    return position_to_index, first_entry

### Algorytm KMR
Służy do zbudowania słownika podstawowych składowych (DBF) dla tekstu. Funkcja zwraca:
- $names$ - słownik list z przyporządkowanymi wartościami $name$ dla kolejnych długości wzorca $2^i$
- $entries$ - słownik słowników; $name \rightarrow$ indeks pierwszego wystąpienia wzorca odpowiadającego $name$

In [131]:
def kmr(text):
    original_length = len(text)
    factor = math.floor(math.log2(original_length))
    if 2**factor < original_length:
        padding_length = 2**(factor + 1) - original_length
        text += unique_char(text) * padding_length
    
    position_to_index, first_entry = sort_rename(text, counting_sort)
    names = {1:position_to_index}
    entries = {1:first_entry}
    for i in range(1,factor):
        power = 2**(i-1)
        new_sequence = []
        for j in range(len(text)):
            if j + power < len(names[power]):
                new_sequence.append((names[power][j], names[power][j + power]))
        position_to_index, first_entry = sort_rename(new_sequence)
        names[power*2] = position_to_index
        entries[power*2] = first_entry
    return names, entries

### Funkcja find_pattern_name
Mając tekst i zbudowany dla niego DBF, funkcja znajduje wartość $name$ odpowiadającą wzorcowi $pattern$, gdzie $|pattern| = 2^i$.

Działanie funkcji korzysta z faktu, iż słownik nazw $name$ zachowuje porządek leksykograficzny wzorców. Dzięki temu można zastosować wyszukiwanie binarne.

Złożoność czasowa: $O(|pattern|\cdot\log|text|)$

In [132]:
def find_pattern_name(text, pattern, dbf):
    patt_len = len(pattern)
    text_len = len(text)
    t = 2**math.floor(math.log2(patt_len))
    if t not in dbf[0]:
        return None
    
    pos = dbf[1][t]
    l, r = 0, len(pos)-1
    pos_idx = (l+r) // 2
    i = 0
    while l <= r:
        curr_positon = pos[pos_idx]
        if text[curr_positon:curr_positon + t] == pattern:
            return pos_idx
        elif text[curr_positon:curr_positon + t] < pattern:
            l = pos_idx + 1
            pos_idx = (l+r) // 2
        else:
            if pos_idx == r:
                return None
            r = pos_idx - 1
            pos_idx = (l+r) // 2
    return None

### Funkcja dbf_search
Realizuje wyszukiwanie wzorca w tekście przy użyciu wyznaczonego wcześniej słownika podstawowych składowych. Dzięki wykorzystaniu funkcji $find\_pattern\_name$ nie jest konieczne tworzenie nowego słownika dla każdego wzorca.

Złożoność czasowa: $O(|text| + |pattern|\cdot\log|text|)$

In [133]:
def dbf_search(text, pattern, dbf=None):
    patt_len = len(pattern)
    text_len = len(text)
    t = 2**math.floor(math.log2(patt_len))
    if dbf is None:
        dbf = kmr(text)     
    
    names = dbf[0][t]
    found = []
    if t == patt_len:
        pattern_name = find_pattern_name(text, pattern[:t], dbf)
        for i in range(patt_len + 1, 2 + text_len):
            if pattern_name == names[i]:
                found.append(i)
    else:
        pattern_name_1 = find_pattern_name(text, pattern[:t], dbf)
        pattern_name_2 = find_pattern_name(text, pattern[-t:], dbf)
        for i in range(patt_len + 1, 2 + text_len):
            if pattern_name_1 == names[i] and pattern_name_2 == names[i + patt_len - t]:
                found.append(i)
    return found

##### Implementacja budowy drzewa sufiksów - algorytm McCreight'a

In [134]:
class SuffNode:
    def __init__(self, tree, start, stop):
        self.start = start
        self.stop = stop
        self.tree = tree
        self.link = None
        self.depth = 0
        self.children = {}
        self.parent = None
        
    def length(self):
        return self.stop - self.start + 1
    
    def label(self):
        return self.tree.text[self.start:self.stop+1]
    
    def letter(self, i):
        return self.tree.text[self.start + i]
    
    def child(self, ch):
        if ch not in self.children:
            return None
        else:
            return self.children[ch]
        
    def add_link(self):
        d = self.depth
        if self.parent == self.tree.root:
            v = self.parent
        else:
            if self.parent.link is None:
                self.parent.add_link()
            v = self.parent.link
            
        offset = 0
        
        if v == self.parent:
            offset = 1
        
        while v.depth < d-1:
            v = v.child(self.tree.text[self.start + v.depth - self.parent.depth + 1])
        if v.depth > d-1:
            v = v.break_path(d-v.parent.depth-1)
        self.link = v
        
    def break_path(self, depth):
        new_node = SuffNode(self.tree, self.start, self.start + depth - 1)
        self.start += depth
        
        self.parent.children[self.tree.text[new_node.start]] = new_node
        new_node.parent = self.parent
        
        new_node.children[self.letter(0)] = self
        self.parent = new_node
        
        new_node.depth = new_node.parent.depth + depth
        
        return new_node
    
    def graft(self, start):
        new_node = SuffNode(self.tree, start, self.tree.text_length-1)
        
        new_node.parent = self
        self.children[self.tree.text[start]] = new_node
        
        return new_node
                

class SuffTree:
    def __init__(self, text):
        self.text = text
        self.text_length = len(text)
        self.root = SuffNode(self, 0, -1)
        self.root.link = self.root
        
        child = SuffNode(self, 0, self.text_length-1)
        child.parent = self.root
        child.depth = self.text_length
        self.root.children[text[0]] = child
            
    def mc_creight(self):
        node = self.root
        self.root.link = self.root
        self.root.depth = 0
        for i in range(1, self.text_length):
            depth = node.depth
            while node.child(self.text[i + depth]):
                node = node.child(self.text[i + depth])
                depth += 1
                node_depth = 1
                while node.start + node_depth <= node.stop and node.letter(node_depth) == self.text[i + depth]:
                    depth += 1
                    node_depth += 1
                if node.start + node_depth <= node.stop and node.letter(node_depth) != self.text[i + depth]:
                    node = node.break_path(node_depth)
                    break
            node.graft(i + depth).depth = self.text_length - i
            if node.link is None:
                node.add_link()
            node = node.link

### Porównanie czasu budowy
- DBF - algorytm KMR
- drzewo sufiksów - algorytm McCreight'a

In [148]:
def kmr_creight_comp(text):
    print("KMR DBF:")
    %timeit kmr(text)
    print("McCreight suffix tree:")
    un = unique_char(text)
    %timeit SuffTree(text + un).mc_creight()
    print("---")

In [203]:
test_filenames = ["romeo-i-julia-700.txt", "1997_714.txt", "zad6"]
test_files = {name: open(name, "r").read() for name in test_filenames}
test_dbfs = {name: kmr(text) for name, text in test_files.items()}

In [210]:
for name, text in test_files.items():
    print(name + ":")
    kmr_creight_comp(text)

romeo-i-julia-700.txt:
KMR DBF:
259 ms ± 7.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
McCreight suffix tree:
37.3 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
---
1997_714.txt:
KMR DBF:
7.48 s ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
McCreight suffix tree:
784 ms ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
---
zad6:
KMR DBF:
11.6 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
McCreight suffix tree:
2.5 ms ± 97.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
---


Wyniki pokazują, że algorytm KMR ma wyższą złożoność obliczeniową $(O(n\log{n}))$ niż algorytm McCreighta $(O(n\log{|\Sigma|}))$, gdzie $\Sigma$ - alfabet

### Porównanie wielkości
- Wielkość pliku tekstowego
- Wielkość zbudowanego słownika podstawowych składowych

In [205]:
def dbf_size(dbf):
    filename = ".sizem.pickle"
    pickle.dump(dbf, open(filename, "wb"))
    size = os.stat(filename).st_size / 1024
    os.remove(filename)
    return size

In [212]:
for name, dbf in test_dbfs.items():
    print(name + ":")
    print(f"File size: {os.stat(name).st_size/1024:.2f} kB")
    print(f"DBF size:  {dbf_size(dbf):.2f} kB")
    print("---")

romeo-i-julia-700.txt:
File size: 13.88 kB
DBF size:  1341.00 kB
---
1997_714.txt:
File size: 248.18 kB
DBF size:  43135.16 kB
---
zad6:
File size: 0.92 kB
DBF size:  54.72 kB
---


Wyniki pokazują wysoką złożoność pamięciową słownika podstawowych składowych $O(n\log{n})$. Zużycie 43 MB pamięci dla tekstu o wielkości 248 kB może być w wielu sytuacjach nieakceptowalne, widzimy więc, że budowanie DBF sprawdza się lepiej dla małych plików

##### Implementacja algorytmu KMP

In [213]:
def kmp_string_matching(text, pattern, pi = None):
    if pi is None:
        pi = prefix_function(pattern)
    patt_len = len(pattern)
    q = 0
    correct_s = []
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            correct_s.append(i - patt_len + 1)
            q = pi[q-1]
    return correct_s
            
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi.append(k)
    return pi

### Porównanie czasu wyszukiwania
- Przy użyciu DBF
- Algorytm KMP

In [225]:
def kmp_dbf_comp(pattern):
    for name, text in test_files.items():
        print(name + ":")
        print("DBF:")
        %timeit dbf_search(text, pattern, test_dbfs[name])
        m1 = dbf_search(text, pattern, test_dbfs[name])
        
        print("KMP:")
        pref = prefix_function(text)
        %timeit kmp_string_matching(text, pattern, pref)
        m2 = kmp_string_matching(text, pattern, pref)
        
        if m1 == m2:
            print(f"Matches found correctly [{len(m1)}]")
        else:
            print("Matching failed")
        print("---")

In [226]:
kmp_dbf_comp("a")

romeo-i-julia-700.txt:
DBF:
559 µs ± 6.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMP:
1.66 ms ± 37.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Matches found correctly [644]
---
1997_714.txt:
DBF:
10.5 ms ± 263 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMP:
30.9 ms ± 395 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Matches found correctly [10346]
---
zad6:
DBF:
40.8 µs ± 981 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
KMP:
119 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Matches found correctly [61]
---


In [ ]:
kmp_dbf_comp("Romeo")